In [386]:
import pandas as pd
import requests
import os

# SALES ETL PROJECT

# EXTRACTION

In [387]:
current_directory = os.getcwd()
file = os.path.join(current_directory, 'sales_products.csv')

In [388]:
current_directory = os.getcwd()
urlSales_DataSource = 'https://62892c727af826e39e69681c.mockapi.io/api/SalesETL/Products'
csvSales__DataSource = pd.read_csv(file)

In [389]:
response = requests.get(urlSales_DataSource)
if(response.status_code==200):
    print('Ok')
else: 
    print('Error')

Ok


### Creating dataframes

In [390]:
url_dataframe = pd.DataFrame(response.json())
csv_dataframe = csvSales__DataSource

In [391]:
url_dataframe.head()

,name,price,description,date,id,brand,nombre,NAME
0,Handcrafted Bronze Car,837.00,New range of formal shirts are designed keepin...,2022-05-22T16:33:56.666Z,1,NaN,NaN,NaN
1,Recycled Fresh Ball,906.00,The Football Is Good For Training And Recreati...,2022-05-21T22:35:17.730Z,2,NaN,NaN,NaN
2,Recycled Rubber Bike,547.00,The automobile layout consists of a front-engi...,2022-05-22T06:55:55.364Z,3,NaN,NaN,NaN
3,Rustic Wooden Computer,129.00,The Football Is Good For Training And Recreati...,2022-05-22T01:18:21.524Z,4,NaN,NaN,NaN
4,Sleek Wooden Chicken,58.00,Carbonite web goalkeeper gloves are ergonomica...,2022-05-22T18:38:39.850Z,5,NaN,NaN,NaN


In [392]:
csv_dataframe.head()

,name,price,description,date,id
0,Bacon,166.0,The Nagasaki Lander is the trademarked name of...,2021-10-26T19:00:24.925Z,1
1,Bike,19.0,Ergonomic executive chair upholstered in bonde...,2021-10-03T18:02:12.519Z,2
2,Towels,216.0,The slim & simple Maple Gaming Keyboard from D...,2022-03-02T21:42:06.848Z,3
3,Hat,464.0,The Nagasaki Lander is the trademarked name of...,2022-05-20T11:46:17.171Z,4
4,Keyboard,819.0,"New ABC 13 9370, 13.3, 5th Gen CoreA5-8250U, 8...",2021-09-02T04:01:05.365Z,5


### Ordering columns

In [393]:
default_column_order = ['id', 'name', 'nombre', 'NAME', 'description', 'price', 'date', 'brand']
url_dataframe = url_dataframe.reindex(columns= default_column_order)

In [394]:
default_column_order = ['id', 'name', 'description', 'price', 'date']
csv_dataframe = csv_dataframe.reindex(columns= default_column_order)

### Concatenating Dataframes

In [395]:
Sales_dataframe = pd.concat([url_dataframe, csv_dataframe],ignore_index=True)
print('url_dataframe rows & columns:' + str(url_dataframe.shape))
print('csv_dataframe rows & columns:' + str(csv_dataframe.shape))
print('Sales_dataframe rows & columns:' + str(Ecommerce_dataframe.shape))
Sales_dataframe.tail(1)

url_dataframe rows & columns:(62, 8)
csv_dataframe rows & columns:(10, 5)
Sales_dataframe rows & columns:(72, 8)


,id,name,nombre,NAME,description,price,date,brand
71,10,Bike,NaN,NaN,The Football Is Good For Training And Recreati...,504.0,2021-11-17T05:18:05.217Z,NaN


#  TRANSFORM

### Removing Unnecessary Columns

In [396]:
Sales_dataframe.isnull().sum()

id              0
name            0
nombre         71
NAME           71
description     0
price           0
date            0
brand          65
dtype: int64

In [397]:
#Search for not nulls indexes
Sales_dataframe.index[Sales_dataframe['nombre'].notnull()]

Int64Index([60], dtype='int64')

In [398]:
Sales_dataframe.index[Sales_dataframe['NAME'].notnull()]

Int64Index([61], dtype='int64')

In [399]:
Sales_dataframe[(Sales_dataframe.id == '61')]

,id,name,nombre,NAME,description,price,date,brand
60,61,Awesome Bronze Ball,charolzapato de,NaN,esto es un zapato de charol de tela bacano,"50,000",2019-05-55 12:00 pm,hecha a mano


In [400]:
Sales_dataframe[(Sales_dataframe.id == '62')]

,id,name,nombre,NAME,description,price,date,brand
61,62,Tasty Granite Chips,NaN,buso,12343,70.00,2019-05-55 12:00 pm,offwhite


### Replacing notna values from nombre and Name columns to name column

In [401]:
nombre_notnull = Sales_dataframe['nombre'].values[60]

In [402]:
NAME_notnull = Sales_dataframe["NAME"].values[61]

In [403]:
for x in range(len(Sales_dataframe.index)):
    if pd.notna(Sales_dataframe['nombre'].values[x]):
        Sales_dataframe['name'].values[x] = Sales_dataframe['name'].values[x].replace(str(Sales_dataframe['name'].values[x]),str(Sales_dataframe['nombre'].values[x]))
    if pd.notna(Sales_dataframe['NAME'].values[x]):
        Sales_dataframe['name'].values[x] = Sales_dataframe['name'].values[x].replace(str(Sales_dataframe['name'].values[x]),str(Sales_dataframe['NAME'].values[x]))

In [404]:
Sales_dataframe[(Sales_dataframe.id == '61')|(Sales_dataframe.id == '62')]

,id,name,nombre,NAME,description,price,date,brand
60,61,charolzapato de,charolzapato de,NaN,esto es un zapato de charol de tela bacano,"50,000",2019-05-55 12:00 pm,hecha a mano
61,62,buso,NaN,buso,12343,70.00,2019-05-55 12:00 pm,offwhite


### Deleting nombre and NAME column

In [405]:
Sales_dataframe = Sales_dataframe.drop(['nombre','NAME'], axis=1)

In [406]:
list(Sales_dataframe.columns)

['id', 'name', 'description', 'price', 'date', 'brand']

### Replacing NaN for 'No Brand' in brand column

In [407]:
Sales_dataframe["brand"]=Sales_dataframe["brand"].fillna('No Brand')

### Checking for mistakes in price column

In [408]:
Sales_dataframe[Sales_dataframe['price'].str.contains(',', na=False)]

,id,name,description,price,date,brand
51,52,gorra,esto es una gorra bacana,"15,000",2022-05-15,No Brand
52,53,pantalon,esto es una pantalon bacano,"$15,000",2022-05-15,No Brand
53,54,chancla,esto es una chancla bacano,"e 15,000",2021/05/15,No Brand
57,58,blue jacket,esto es una chancla bacano,"$ 50,000",2021/05/15 12:00 pm,aeropostal
58,59,ruana,esto es una chancla bacano,"$ 15,000",2019-05-55 12:00 pm,aeropostal
59,60,ruana de tela,esto es una ruana de tela bacano,"45,00",2019-05-55 12:00 pm,hecha a mano
60,61,charolzapato de,esto es un zapato de charol de tela bacano,"50,000",2019-05-55 12:00 pm,hecha a mano


### Fixing mistakes

In [409]:
Sales_dataframe["price"]=Sales_dataframe["price"].astype(str).str.replace(",", ".")
Sales_dataframe["price"]=Sales_dataframe["price"].str.replace("€", "")
Sales_dataframe["price"]=Sales_dataframe["price"].str.replace("$", "")
Sales_dataframe["price"]=Sales_dataframe["price"].str.replace("e", "")
Sales_dataframe["price"]=Sales_dataframe["price"].str.replace(" ", "")

C:\Users\lizme\AppData\Local\Temp\ipykernel_11832\193647780.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Sales_dataframe["price"]=Sales_dataframe["price"].str.replace("$", "")


In [410]:
Sales_dataframe[Sales_dataframe['price'].str.contains(',', na=False)]
Sales_dataframe[Sales_dataframe['price'].str.contains('€', na=False)]
Sales_dataframe[Sales_dataframe['price'].str.contains('$', na=False)]
Sales_dataframe[Sales_dataframe['price'].str.contains('e', na=False)]
Sales_dataframe[Sales_dataframe['price'].str.contains(' ', na=False)]

,id,name,description,price,date,brand


### Deleting dates out of range

In [411]:
Sales_dataframe[Sales_dataframe['date'].str.contains('2019-05-55 12:00 pm', na=False)]

,id,name,description,price,date,brand
58,59,ruana,esto es una chancla bacano,15.000,2019-05-55 12:00 pm,aeropostal
59,60,ruana de tela,esto es una ruana de tela bacano,45.00,2019-05-55 12:00 pm,hecha a mano
60,61,charolzapato de,esto es un zapato de charol de tela bacano,50.000,2019-05-55 12:00 pm,hecha a mano
61,62,buso,12343,70.00,2019-05-55 12:00 pm,offwhite


In [412]:
Sales_dataframe.drop([58,59,60,61], axis=0, inplace=True)

### Standarize date column

In [413]:
Sales_dataframe.dtypes

id             object
name           object
description    object
price          object
date           object
brand          object
dtype: object

In [414]:
Sales_dataframe['date'] = pd.to_datetime(Sales_dataframe['date'])
Sales_dataframe['date'] = pd.to_datetime(Sales_dataframe['date'], utc=True, format='%Y-%m-%d')
Sales_dataframe['date'] = Sales_dataframe['date'].dt.date

In [415]:
Sales_dataframe.head(5)

,id,name,description,price,date,brand
0,1,Handcrafted Bronze Car,New range of formal shirts are designed keepin...,837.00,2022-05-22,No Brand
1,2,Recycled Fresh Ball,The Football Is Good For Training And Recreati...,906.00,2022-05-21,No Brand
2,3,Recycled Rubber Bike,The automobile layout consists of a front-engi...,547.00,2022-05-22,No Brand
3,4,Rustic Wooden Computer,The Football Is Good For Training And Recreati...,129.00,2022-05-22,No Brand
4,5,Sleek Wooden Chicken,Carbonite web goalkeeper gloves are ergonomica...,58.00,2022-05-22,No Brand


### Fixing Index

In [416]:
Sales_dataframe = Sales_dataframe.reset_index(drop=True)
for i in Sales_dataframe.index:
    Sales_dataframe["id"][i]=i+1
    i+=1

In [417]:
Sales_dataframe = Sales_dataframe.reset_index(drop=True)

# LOAD

In [421]:
Sales_dataframe.to_csv(current_directory)

PermissionError: [Errno 13] Permission denied: 'C:\\Academic Projects\\Sales ETL Project'